# Motor files relocation

The ideia is to relocate all the motors recovery files based on their's creation date. This creation date will be get from Traceability system database based on the day their passed on Reflash station.

In [1]:
import pandas as pd
import os
from IPython.display import display as print_dataframe

## Loading the file lists

In [2]:
files_list_folder = 'C:\\Users\\BZLJ0R\\OneDrive - NEXTEER AUTOMOTIVE\\Documents\\motor_recovery\\files_lists'

In [3]:
df = pd.DataFrame()

for file_list in os.listdir(files_list_folder):
    print(file_list)
    
    df_temp = pd.read_csv(os.path.join(files_list_folder, file_list)
                         ,sep = ";"
                         ,header=None
                         ,names=['file']
                         ,engine="python")
    
    #df_temp.rename(columns = {'0': 'file'}, inplace = True)
    df_temp['list_name'] = file_list.replace('.txt','')
    
    if df.empty:
        df = df_temp
    else:
        df = pd.concat([df,df_temp])

antigos.txt
REFLASH_1.txt
REFLASH_2.t.txt
REFLASH_2REFLASH_2.txt
REFLASH_3.txt
REFLASH_4.txt
REFLASH_6.txt


In [5]:
df.shape

(1583189, 2)

In [6]:
df.head(2)

,file,list_name
0,Inhale SD000133X01 Q007406486 11-Aug-20 6_18_...,antigos
1,Inhale SD000133X01 Q008305121 27-Sep-20 16_59...,antigos


## Extracting Motor Serial Number information from file name

In [7]:
# filtering motor files. Are the ones with " Inhale" on their names.

df['inhale_string_start_position'] = df['file'].str.find(' Inhale')

records_with_Inhale_string_in_file_name = df['inhale_string_start_position'] > -1

df = df[records_with_Inhale_string_in_file_name]

In [8]:
# extracting the position of .xml string to after that split file name on desired fields

df['xml_string_start_position'] = df['file'].str.find('.xml')

In [10]:
df.head(2)

,file,list_name,inhale_string_start_position,xml_string_start_position
0,Inhale SD000133X01 Q007406486 11-Aug-20 6_18_...,antigos,0,48
1,Inhale SD000133X01 Q008305121 27-Sep-20 16_59...,antigos,0,49


In [11]:
def slice_str(string, start, stop):
    return string[start:stop].strip()

df['file_name_substring'] = df.apply(
                           lambda x:  slice_str(
                                                str(x[0])
                                               ,x['inhale_string_start_position']
                                               ,x['xml_string_start_position']
                                               )
                                      ,axis = 1
                          )

In [12]:
def remove_am_pm_mark(string):
    return string.replace(' AM','').replace(' PM','')

df['file_name_formated'] = df['file_name_substring'].apply(remove_am_pm_mark)

In [13]:
df.drop(columns = ['inhale_string_start_position','xml_string_start_position'], inplace = True)
print_dataframe(df.head(2))

,file,list_name,file_name_substring,file_name_formated
0,Inhale SD000133X01 Q007406486 11-Aug-20 6_18_...,antigos,Inhale SD000133X01 Q007406486 11-Aug-20 6_18_10,Inhale SD000133X01 Q007406486 11-Aug-20 6_18_10
1,Inhale SD000133X01 Q008305121 27-Sep-20 16_59...,antigos,Inhale SD000133X01 Q008305121 27-Sep-20 16_59_12,Inhale SD000133X01 Q008305121 27-Sep-20 16_59_12
